In [ ]:
import numpy as np
import pandas as pd
import tensorflow_probability as tfp

from meridian import constants
from meridian.data import data_frame_input_data_builder as data_builder
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution

from mmm_eval import (
    MeridianConfig, MeridianInputDataBuilderSchema, run_evaluation)

import meridian

## Load your data

The data below comes from the Meridian repository and gives a great example of how the data should be formatted.

In [ ]:
df = pd.read_excel(
    'https://github.com/google/meridian/raw/main/meridian/data/simulated_data/xlsx/geo_media.xlsx',
    engine='openpyxl',
)

In [ ]:
builder = (
    data_builder.DataFrameInputDataBuilder(kpi_type='non_revenue')
        .with_kpi(df, kpi_col="conversions")
        .with_revenue_per_kpi(df, revenue_per_kpi_col="revenue_per_conversion")
        .with_population(df)
        .with_controls(df, control_cols=["GQV", "Discount", "Competitor_Sales"])
)
channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4", "Channel5"]
builder = builder.with_media(
    df,
    media_cols=[f"{channel}_impression" for channel in channels],
    media_spend_cols=[f"{channel}_spend" for channel in channels],
    media_channels=channels,
)

data = builder.build()

## Define a Meridian MMM

In [ ]:
roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)
# sampling from the posterior is not required prior to evaluation
mmm = model.Meridian(input_data=data, model_spec=model_spec)

# I wonder if this MMM is any good... let's check!

First, you will need to specify how the MMM should be built to match the `DataFrameInputDataBuilder` logic above.

See the [Meridian data loading documentation](https://developers.google.com/meridian/docs/user-guide/supported-data-types-formats?hl=en) if you need more guidance.

### Preprocess data

We preprocess the data in two ways:

- calculate revenue from "revenue_per_conversion" as `mmm-eval` expects revenue, not revenue per KPI
- restricting dates and geos to reduce runtime

In [ ]:
data_preproc = df.copy()
data_preproc["revenue"] = data_preproc["revenue_per_conversion"]*data_preproc["conversions"]

# restrict to only two geos
data_preproc = data_preproc[data_preproc["geo"].isin(["Geo0", "Geo1"])]

# restrict to only post-2023
data_preproc = data_preproc[pd.to_datetime(data_preproc["time"]) > pd.Timestamp("2023-01-01")]

In [ ]:
channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4", "Channel5"]
input_data_builder_config = MeridianInputDataBuilderSchema(
    date_column="time",
    media_channels=channels,
    channel_spend_columns=[f"{col}_spend" for col in channels],
    channel_impressions_columns=[f"{col}_impression" for col in channels],
    response_column="conversions",
    control_columns=["GQV", "Competitor_Sales", "Discount"],
)

In [ ]:
# specify a larger number of samples if you want quality results
sample_posterior_kwargs = dict(n_chains=1, n_adapt=10, n_burnin=10, n_keep=10)
config = MeridianConfig.from_model_object(mmm, input_data_builder_config=input_data_builder_config,
                                          revenue_column="revenue", sample_posterior_kwargs=sample_posterior_kwargs)

In [ ]:
# Run the evaluation suite!
result = run_evaluation(framework="meridian", config=config, data=data_preproc)

## Examine the results

In [ ]:
# Let's see what we got
display(result)